In [55]:
import pandas as pd
import spacy
from time import time
from gensim.models import Word2Vec
import logging
import numpy as np
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk import ngrams

# Arrumando nosso DataSet de drogas

In [56]:
#Pegando o dataset de drogas
dados_treino = pd.read_csv("../Data/resultado_drogas.csv", encoding="latin-1", sep=";", dtype="unicode")
dados_treino.tail()

,CHAVE_OBJETO,ANO_OCOR,NRO_OCOR,NRO_INT_TIPO_OBJETO,TXT_TIPO_OBJETO,Cor Predominante,Descrição,Peso Total,Peso Unitário,Quantidade de Unidade(s),Tipo de Embalagem,Unidade de Peso,DESC_PADRAO,RECLASSIFICAÇÃO
113226,24126969,2021,2148,5,Maconha,Verde,NaN,"415,00",NaN,1,Porção(es),Grama,NaN,Maconha
113227,24126972,2021,2148,5,Maconha,Verde,NaN,"5,00",NaN,1,Porção(es),Grama,NaN,Maconha
113228,24126984,2021,8335,756,Cocaína,Branco,aproximadamente 23 gramas de pinos de cocaína,NaN,NaN,36,Unidade(s),Grama,aproximadamente 23 gramas de pinos de cocaina,Cocaína
113229,24127010,2021,14951,5,Maconha,Verde,Um tijolo de erva verde semelhante a maconha p...,"642,00",NaN,1,Tijolo(s),Grama,um tijolo de erva verde semelhante a maconha p...,Maconha
113230,24127011,2021,14951,5,Maconha,Verde,Um tijolo de erva verde semelhante a maconha p...,"1058,00",NaN,1,Tijolo(s),Grama,um tijolo de erva verde semelhante a maconha p...,Maconha


In [57]:
# Criando objeto spacy 
nlp = spacy.load("pt_core_news_lg")

In [58]:
# Função transforma o texto recebido em um objeto nlp para que possamos usar os pipes da spacy para 
# trata-lo, tirando as palavras classificadas como Stop Words.
# As demais palavras são guardadas na lista 'token_validos'
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        valido = not token.is_stop and token.is_alpha
        if valido:
            tokens_validos.append(token.text)
            
    if len(tokens_validos) > 2:
        return " ".join(tokens_validos) # Cada token é retornado com um espaço entre eles

In [59]:
textos_para_tratamento = (str(descricoes).lower() for descricoes in dados_treino["Descrição"]) # Generator Expression
textos_para_tratamento

<generator object <genexpr> at 0x000001F187FF5CF0>

In [60]:
# 'texto_tratados' recebe o retorno da função trata_textos que por sua vez recebe um objeto nlp 
# que vem da comprehension list 'textos_para_tratamento'
# Também estamos verificando o tempo que leva para executar a função 'time'
tempo_inicial = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                         batch_size = 120000,
                                                         n_process = -1)]

tempo_final = time() - tempo_inicial

print(tempo_final/60)

9.441034909089407


In [61]:
descricoes_tratadas = pd.DataFrame({"descricao": textos_tratados,
                                    "classificacao":dados_treino.RECLASSIFICAÇÃO})
descricoes_tratadas.tail()

,descricao,classificacao
113226,None,Maconha
113227,None,Maconha
113228,aproximadamente gramas pinos cocaína,Cocaína
113229,tijolo erva verde semelhante maconha pesanto a...,Maconha
113230,tijolo erva verde semelhante maconha pesando a...,Maconha


In [62]:
#Retirando do DataFrame as linhas nulas e as duplicadas
# Tamanho antes
print(len(descricoes_tratadas))

descricoes_tratadas = descricoes_tratadas.dropna().drop_duplicates()
# Tamanho depois
print(len(descricoes_tratadas))

113231
51697


In [63]:
descricoes_tratadas.to_csv("../Data/df_procergs.csv", encoding="latin-1", sep=";",index=False)


# Construindo o vocabulário

In [64]:
# Para construir o nosso vocabulário, precisamos de uma lista de lista de tokens para
# usar no Word2Vec
lista_lista_tokens = [desc.split(' ') for desc in descricoes_tratadas.descricao]

In [65]:
#sg = 0 --> Modelo cbow (Recebe contexto, prevê palavra)
#sg = 1 --> Modelo SkipGram (Recebe palavra, prevê contexto)
#window = quantas palavras ele leva em consideração antes e quantas depois
#vector_size = dimenões do vetor
#min_count = mínimo de vezes que uma palavra deve aparecer para ser considerada
#alpha = valor que determina qual a taxa de aprendendizado e a de redução da função de custo
#min_alpha = valor mínimo da distancia que a função de custo deve decair

# Logging mostra o progresso na tela
logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 5,
                      vector_size = 300,
                      min_count = 0,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per = 5000)

2021-10-04 16:18:10,862 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2021-10-04T16:18:10.859892', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2021-10-04 16:18:10,901 : - collecting all words and their counts
2021-10-04 16:18:10,903 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-04 16:18:10,917 : - PROGRESS: at sentence #5000, processed 29277 words, keeping 1822 word types
2021-10-04 16:18:10,933 : - PROGRESS: at sentence #10000, processed 60106 words, keeping 2813 word types
2021-10-04 16:18:10,946 : - PROGRESS: at sentence #15000, processed 92131 words, keeping 3740 word types
2021-10-04 16:18:10,961 : - PROGRESS: at sentence #20000, processed 125251 words, keeping 4473 word types
2021-10-04 16:18:10,975 : - PROGRESS: at sentence #25000, processed 155383 w

In [66]:
# Contando o número de linhas do corpus criado no vocabulário
w2v_modelo.corpus_count

51697

# Treinamento do modelo Word2Vec

In [67]:
# 1o parâmetro: sentenças que queremos passar
# 2o parâmetro: total de amostras
# 3o parâmetro: épocas do nosso treinamento
# OBS: A cada época, o modelo calcula o loss (curva de aprendizado) e atualiza
# o modelo n vezes (neste caso, 30)

w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30)

2021-10-04 16:18:11,341 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 8018 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2021-10-04T16:18:11.341261', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'train'}
2021-10-04 16:18:11,891 : - worker thread finished; awaiting finish of 2 more threads
2021-10-04 16:18:11,892 : - worker thread finished; awaiting finish of 1 more threads
2021-10-04 16:18:11,896 : - worker thread finished; awaiting finish of 0 more threads
2021-10-04 16:18:11,897 : - EPOCH - 1 : training on 332305 raw words (169125 effective words) took 0.5s, 356808 effective words/s
2021-10-04 16:18:12,377 : - worker thread finished; awaiting finish of 2 more threads
2021-10-04 16:18:12,380 : - worker thread finished; awaiting finish of 1 more threads
2021-10-04 1

(5072777, 9969150)

# Testando o métodod 'most_similar'

In [68]:
w2v_modelo.wv.most_similar("cocaina")

[('cocaína', 0.7142294049263),
 ('cocaíana', 0.4394259750843048),
 ('cocaínaa', 0.40420371294021606),
 ('cocaíca', 0.3886478841304779),
 ('cacaína', 0.3858521580696106),
 ('cacaina', 0.37731990218162537),
 ('côr', 0.37397757172584534),
 ('cocainaa', 0.3711448013782501),
 ('eppenford', 0.36748018860816956),
 ('po', 0.3665960729122162)]

In [69]:
w2v_modelo.wv.most_similar("maconha")

[('macoha', 0.4754354655742645),
 ('prensada', 0.4708314538002014),
 ('maconhanha', 0.45339810848236084),
 ('maconhaha', 0.44962409138679504),
 ('cannabis', 0.4407317638397217),
 ('artesanais', 0.4379664957523346),
 ('macona', 0.41701069474220276),
 ('erva', 0.41121476888656616),
 ('seca', 0.408692866563797),
 ('castanho', 0.40836042165756226)]

In [70]:
w2v_modelo.wv.most_similar("crack")

[('crak', 0.651922345161438),
 ('carck', 0.3957770764827728),
 ('rosário', 0.3905513882637024),
 ('craque', 0.3871470093727112),
 ('várias', 0.35703083872795105),
 ('crck', 0.3518581688404083),
 ('sólida', 0.34381231665611267),
 ('pepesando', 0.34373044967651367),
 ('cracka', 0.33842140436172485),
 ('alumínio', 0.3345668911933899)]

# Salvando o modelo

In [71]:
#w2v_modelo_sg.wv.save_word2vec_format('../Models/modelo_sg_drogas.txt', binary=False)
#w2v_modelo.wv.save_word2vec_format('../Models/modelo_w2v_drogas.txt', binary=False)

# Desabiltando certos pipes fazem o modelo rodar mais rápido

In [72]:
#parser é um pipe analisador de dependência sintática da spacy, ou seja, atribui em cada palavra um label que descreve sua função na frase
#ner é um pipe de reconhecimento de entidades da spacy, permitindo que cada palavra tenha um label como "PER" (pessoa) ou "LOC" (localização), etc
#tagger é um pipe da spacy identificador de classe gramatical da língua carregada no load (neste caso, português)
#textcat é um pipe da spacy que é um categorizador de texto. Ele prevê categorias em um documento inteiro.
nlp = spacy.load('pt_core_news_lg', disable=['parser','ner','tagger','textcat'])

def trata_textos_rapida(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    return tokens_validos

texto = "42 gramas de maconha apreendidos"
tokens = trata_textos_rapida(texto)
print(tokens)

['gramas', 'maconha', 'apreendidos']


# Carregando nosso modelo Word2Vec

In [73]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format('../Models/modelo_w2v_drogas.txt')

2021-10-04 16:18:34,189 : - loading projection weights from ../Models/modelo_w2v_drogas.txt
2021-10-04 16:18:35,706 : - KeyedVectors lifecycle event {'msg': 'loaded (8018, 300) matrix of type float32 from ../Models/modelo_w2v_drogas.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-10-04T16:18:35.706957', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'load_word2vec_format'}


# Vetorizando as palavras para o nosso modelo

In [74]:
# 'vetor_resultante' é a quantidade de dimensões que o vetor tem (no caso, 300)
def combinacao_de_vetores_por_soma(palavras, modelo):

    # Como tokens é o exemplo "42 gramas de maconha apreendidos"
    # São 300 dimensões para uma única linha
    vetor_resultante = np.zeros((1, 300))
    # Para cada token das palavras
    for p in palavras:    
        try: # Pegamos o vetor deste token e somamos
             # no 'vetor_resultante'
            vetor_resultante += modelo.get_vector(p)

        except KeyError:
           pass

    return vetor_resultante


vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[ 0.1286815  -1.90977759  0.13999526  1.59192276 -0.75647955 -0.01888183
   0.09412077 -0.1529047  -0.17223457  0.0638263  -0.10664336 -0.7182117
  -0.54101728 -0.59995277 -0.22309858  0.45804346 -0.06492934 -0.66322355
  -0.02955554  0.446263    1.27539738  1.01556657 -1.0327418   0.23260039
   1.17448529 -1.13782222  1.96738273  0.17222127  0.85432186 -0.08431077
   0.36445507  1.25217181  0.28909496 -0.81386827 -0.44321911 -0.87905075
   0.85559136 -1.61212209 -0.21839337 -0.31124746 -0.12727931 -0.31963331
   1.21662232  0.37635526  0.59015531  0.92955929 -1.36795782  0.10052644
   0.21649069 -0.64006384  0.45780528  0.59258001  0.2170861   0.45159146
  -0.76111618 -0.24482659  0.12455752 -0.57386573 -0.77144793  0.06450007
   0.4309913   0.28348443  0.09103077 -2.0773297  -0.12554504 -0.07390217
  -0.80918305 -0.30200598  0.28753987 -0.44278804  1.19332951 -1.11024354
  -0.89336275 -0.75083446  0.70570676 -0.1603111  -0.12427431  0.53098689
  -0.00523202 -0.04001921  1.5

In [75]:
def matriz_vetores(textos, modelo):
    #Numero de linhas da matriz depende do numero de linhas que serão vetorizadas
    x = len(textos)
    #Numero de dimensões
    y = 300
    matriz = np.zeros((x,y))

    #Vetorizarando o texto
    for i in range(x):
        palavras = trata_textos_rapida(str(textos.iloc[i])) # i representa o index no dataset, ou seja, para cada descrição neste index
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz


In [76]:
descricoes_tratadas

,descricao,classificacao
50,pequena porção erva cor esverdeada semelhante ...,Maconha
51,substancia semelhanta crack,Crack
52,substancia semelhante cocaina,Cocaína
55,aprox gramas embalagem pinos,Drogas
57,pote contendo pequena quantidade substancia es...,Drogas
...,...,...
113218,porção substância características maconha invó...,Maconha
113219,porção substância características maconha invó...,Maconha
113222,pequenas pedras crack pedra mesma substância p...,Crack
113228,aproximadamente gramas pinos cocaína,Cocaína


In [77]:
tamanho_treino = len(descricoes_tratadas) * 0.8 # 80% das descrições
tamanho_teste = len(descricoes_tratadas) * 0.2 # 20% das descrições

matriz_vetores_treino_cbow = matriz_vetores(descricoes_tratadas.descricao.head(int(tamanho_treino)), w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(descricoes_tratadas.descricao.tail(int(tamanho_teste)), w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(41357, 300)
(10339, 300)


# Utilizando a técnica TF-IDF

In [78]:
def classificador(modelo, x_treino, y_treino, x_teste, y_teste):


    # Usando modelo de Regressão Logística
    RL = LogisticRegression(max_iter  = 800)
    RL.fit(x_treino, y_treino)
    
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    
    print(resultados)
    return RL

In [79]:
# 'max_features' é o número de vezes que a palavra apareceu para ser considerada (frequência dela)
frases =['Assisti um filme ótimo', 'Assisti um  filme péssimo']

tfidf = TfidfVectorizer(lowercase=False, max_features=50)

caracteristicas = tfidf.fit_transform(frases)
pd.DataFrame(
        caracteristicas.todense(),
        columns =  tfidf.get_feature_names()
)

,Assisti,filme,péssimo,um,ótimo
0,0.448321,0.448321,0.000000,0.448321,0.630099
1,0.448321,0.448321,0.630099,0.448321,0.000000


In [80]:
# Tirando linhas duplicadas e nulas
dados_treino_not_na = descricoes_tratadas.dropna()
dados_treino_drop_dup = dados_treino_not_na.drop_duplicates()
dados_treino_drop_dup.shape

(51697, 2)

In [81]:
dados_treino_drop_dup

,descricao,classificacao
50,pequena porção erva cor esverdeada semelhante ...,Maconha
51,substancia semelhanta crack,Crack
52,substancia semelhante cocaina,Cocaína
55,aprox gramas embalagem pinos,Drogas
57,pote contendo pequena quantidade substancia es...,Drogas
...,...,...
113218,porção substância características maconha invó...,Maconha
113219,porção substância características maconha invó...,Maconha
113222,pequenas pedras crack pedra mesma substância p...,Crack
113228,aproximadamente gramas pinos cocaína,Cocaína


In [82]:
dados_treino_drop_dup

,descricao,classificacao
50,pequena porção erva cor esverdeada semelhante ...,Maconha
51,substancia semelhanta crack,Crack
52,substancia semelhante cocaina,Cocaína
55,aprox gramas embalagem pinos,Drogas
57,pote contendo pequena quantidade substancia es...,Drogas
...,...,...
113218,porção substância características maconha invó...,Maconha
113219,porção substância características maconha invó...,Maconha
113222,pequenas pedras crack pedra mesma substância p...,Crack
113228,aproximadamente gramas pinos cocaína,Cocaína


### Instanciando modelo de RL sem os n-grams

In [83]:
regressaao_logistica1 = LogisticRegression(max_iter=100000)

vetor_tf_idf_bruto =  tfidf.fit_transform(dados_treino_drop_dup.descricao)
treino, teste, classe_treino, classe_teste = train_test_split(vetor_tf_idf_bruto, # x
                                                               dados_treino_drop_dup.classificacao, # y
                                                               random_state=42)


regressaao_logistica1.fit(treino,classe_treino)
acuraria_vetor_tf_idf_bruto = regressaao_logistica1.score(teste, classe_teste)
print(acuraria_vetor_tf_idf_bruto)

0.9318375241779497


### Instanciando modelo RL utilizando TF-IDF com n-grams

In [84]:
                                                     #bi-gram
tfidf = TfidfVectorizer(lowercase=False, ngram_range  = (1,2))

vetor_tfidf = tfidf.fit_transform(dados_treino_drop_dup.descricao)
treino, teste, classe_treino, classe_teste = train_test_split(vetor_tfidf, 
                                                               dados_treino_drop_dup.classificacao,
                                                               random_state=42)

regressaao_logistica2 = LogisticRegression(max_iter=100000)


regressaao_logistica2.fit(treino, classe_treino)
acuraria_vetor_tf_idf_ngrams = regressaao_logistica2.score(teste, classe_teste)
print(acuraria_vetor_tf_idf_ngrams)

0.942514506769826


In [85]:
#coef_[2] -> Drogas
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[2].T,
    index =  tfidf.get_feature_names(),
)

pesos.nlargest(10,0)

,0
comprimidos,5.132208
vinculado,4.873380
ecstasy,4.146195
lsd,3.030774
frasco,2.964612
frascos,2.951568
mdma,2.502555
comprimido,2.398119
cod,2.180347
haxixe,2.069378


In [86]:
#coef_[0] -> Cocaína
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[0].T,
    index =  tfidf.get_feature_names(),
)

pesos.nlargest(10,0)

,0
cocaína,18.295910
cocaina,13.682737
pinos,5.497896
cocaína pesando,3.765937
cocaina pesando,3.370358
semelhante cocaína,3.366675
pó,3.126336
buchas,2.845291
pino,2.606420
características cocaína,2.297428


In [87]:
#coef_[1] -> Crack
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[1].T,
    index =  tfidf.get_feature_names(),
)

pesos.nlargest(10,0)

,0
crack,20.817663
pedras,6.984995
pedrinhas,4.455940
pedra,4.077910
crak,4.074462
crack pesando,4.033724
semelhante crack,3.755415
amarelada,2.561995
características crack,2.022532
crack peso,1.960310


In [88]:
#coef_[3] -> Maconha
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[3].T,
    index =  tfidf.get_feature_names(),
)

pesos.nlargest(10,0)

,0
maconha,23.274256
semelhante maconha,4.528759
maconha pesando,4.431737
cigarro,3.479635
sativa,2.657055
cannabis,2.472395
esverdeada,2.441893
gramas maconha,2.378084
erva,2.364988
tijolos,2.286584


In [89]:
# coef_[3] -> Maconha ou seja, os menores coeficientes para maconha.
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[3].T,
    index =  tfidf.get_feature_names(),
)

pesos.nsmallest(10,0)

,0
crack,-8.206391
cocaína,-7.804743
cocaina,-5.807259
pedras,-4.423848
pinos,-4.203907
pó,-3.117448
pedra,-3.064535
pedrinhas,-2.662215
comprimidos,-2.538912
branca,-2.190868


In [90]:
# coef_[2] -> Drogas ou seja, os menores coeficientes para drogas.
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[2].T,
    index =  tfidf.get_feature_names(),
)

pesos.nsmallest(10,0)

,0
maconha,-9.621552
cocaína,-6.567467
crack,-6.460063
cocaina,-5.012867
semelhante maconha,-2.452086
maconha pesando,-2.163029
perícia,-1.886364
odor,-1.418984
crack pesando,-1.417142
cocaína pesando,-1.415791


In [91]:
# coef_[1] -> Crack ou seja, os menores coeficientes para crack.
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[1].T,
    index =  tfidf.get_feature_names(),
)

pesos.nsmallest(10,0)

,0
maconha,-6.752495
cocaína,-3.923700
cocaina,-2.862611
cigarro,-1.876182
pinos,-1.864810
maconha pesando,-1.859262
esverdeada,-1.855194
pó,-1.784946
erva,-1.741176
semelhante maconha,-1.461552


In [92]:
# coef_[0] -> Ccaína, ou seja, os menores coeficientes para cocaína.
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[0].T,
    index =  tfidf.get_feature_names(),
)

pesos.nsmallest(10,0)

,0
maconha,-6.900210
crack,-6.151209
erva,-2.509346
esverdeada,-2.503634
cigarro,-2.367941
pedras,-2.325840
papel,-1.594520
pedrinhas,-1.520630
vinculado,-1.495969
cigarros,-1.468609


# Teste de frases para o modelo

In [147]:
vetor_tfidf_teste_maconha = tfidf.transform(['Uma porção de maconha', '40g de erva verde em uma bolsa'])
vetor_tfidf_teste_cocaina = tfidf.transform(['7 pinos com aprox. 20g', 'Saquinhos de pó achados em uma loja'])
vetor_tfidf_teste_crack = tfidf.transform(['2 pedras de tom amarelado', '6 unidades de crac.'])
vetor_tfidf_teste_drogas = tfidf.transform(['Bolsa apreendida com lsd', 'mdma'])

In [112]:
regressaao_logistica2.predict(vetor_tfidf_teste_maconha)

array(['Maconha', 'Maconha'], dtype=object)

In [149]:
regressaao_logistica2.predict(vetor_tfidf_teste_cocaina)

array(['Cocaína', 'Cocaína'], dtype=object)

In [148]:
regressaao_logistica2.predict(vetor_tfidf_teste_crack)

array(['Crack', 'Crack'], dtype=object)

In [150]:
regressaao_logistica2.predict(vetor_tfidf_teste_drogas)

array(['Drogas', 'Drogas'], dtype=object)